In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from dataloader import SplicingKmerDataset

In [18]:
from kipoi.data import DataLoader
from kipoi.data_utils import numpy_collate

In [19]:
from model import HALModel

In [20]:
fasta = "/s/genomes/human/hg19/ensembl_GRCh37.p13_release75/Homo_sapiens.GRCh37.75.dna.primary_assembly.fa"
GFF = "/s/genomes/human/hg19/MISO/A5SS.hg19.nochr.selected.gff3"

In [21]:
data = SplicingKmerDataset(GFF, 
                           fasta,
                           MISO_AS=True) #get_A5SS_dt, otherwise return a dicionary with inputs, targets...

In [22]:
test = data[2]

test

{'inputs': array([ 'CCGCGCTACTTCGGCCCCAACTTGCTCAACACGGTGAAGCAGAAGCTCTTCACCGAGGTGGAGGGGACCTGCACAGGGAAGTGAGTGTCGAGCTCACCGCACCGCCAGATCGTTCGATGCGCACACGGGGCGCTATACCTGCAACCCCTCCCCAACTCCT',
        'GAGATGGGTCATTCATTGACTCCTCAAACATTTTAAAAACCCTTGGTGTGTGCCAGGACCCGTGTTGGGCTCAGAACAAGGTAGAATGGGAACAAGATAGAATGAGGACAGCCCCTGTGGAGCTCACAGTGTAATCATTGTTCCTGTGTCTGCTCTCCCT'],
       dtype='<U160'),
 'metadata': {'chrom': '11',
  'geneName': 'chr11:62529267:62529376|62529656:+@chr11:62530339:62530498:+',
  'start': 62529267,
  'stop': 62530498,
  'strand': '+'}}

In [23]:
seqs = test['inputs']

In [24]:
seqs[0][-80:]

'GTGAGTGTCGAGCTCACCGCACCGCCAGATCGTTCGATGCGCACACGGGGCGCTATACCTGCAACCCCTCCCCAACTCCT'

In [25]:
seqs[0][80:]

'GTGAGTGTCGAGCTCACCGCACCGCCAGATCGTTCGATGCGCACACGGGGCGCTATACCTGCAACCCCTCCCCAACTCCT'

## Prediction

In [26]:
data_generator = DataLoader(data, batch_size=3, collate_fn=numpy_collate, shuffle=True)
data_generator = iter(data_generator)

In [27]:
data = next(data_generator)

In [28]:
data['inputs']

array([[ 'TTAGCTGTGGAACAAGAATTAGCACAAGAAGAAGAAGAAGAATCAGGACAAGAAGAGCCAAGGGGCCCAGATGGAGACTGGTTAAACACTTATTTCCTTTGCTGTTGGGAGCTCAAAGATGATGTGTTTACACGGCTTACAGTGAAACCTTTGTAGGCTC',
        'GGAGACTGGTTAAACACTTATTTCCTTTGCTGTTGGGAGCTCAAAGATGATGTGTTTACACGGCTTACAGTGAAACCTTTGTAGGCTCATGCTTTATTCCAGAATGATGGAATGCACTGCACCTTCAGGAAGCTGTGCTAGGAAAAGTCTTTCCTTGGTC'],
       [ 'TACACTACCTACAAGATCCCGCGTCAGCGGATTATGGATTACTATGAGACCAACAGCCAGTGCTCCAAGCCCGGAATTGTGTAGGTGGTACACACACATCACACTGGGGGGAGAGGGAGCCAGCAGGGCCTCCTGGAGGGAAGCAGGGAGTGGTGGTGGA',
        'CTACCTACAAGATCCCGCGTCAGCGGATTATGGATTACTATGAGACCAACAGCCAGTGCTCCAAGCCCGGAATTGTGTAGGTGGTACACACACATCACACTGGGGGGAGAGGGAGCCAGCAGGGCCTCCTGGAGGGAAGCAGGGAGTGGTGGTGGAATGG'],
       [ 'CAAGTCCAACACACCAACCCCAAGGAACGACGCCCCAACTCCAGGCACCAGCACGACCCCAGGGCTCAGGTCGATGCCGGGTAAACCTCCGGGCATGGACCCGATAGGTATAATGGGTAGGCACCATGGGCTGGGTTAATCTGATTGGGGGGAGGGGGTC',
        'AACTCCAGGCACCAGCACGACCCCAGGGCTCAGGTCGATGCCGGGTAAACCTCCGGGCATGGACCCGATAGGTATAATGGGTAGGCACCATGGGCTGGGTTAATCTGATTGGGGGGAGGGGGTCTG

In [29]:
HAL = HALModel()

In [30]:
HAL.predict_on_batch(next(data_generator))

array([  1.10469418e-03,   8.81532781e-01,   5.08689968e-05])

# With normal ensembl gtf annotation

In [31]:
fasta = "/s/genomes/human/hg19/ensembl_GRCh37.p13_release75/Homo_sapiens.GRCh37.75.dna.primary_assembly.fa"
GFF = "/s/genomes/human/hg19/ensembl_GRCh37.p13_release75/Homo_sapiens.GRCh37.75.chr22.gtf"

In [32]:
data = SplicingKmerDataset(GFF, 
                           fasta,
                           MISO_AS=False)

In [33]:
data[0]

{'inputs': 'TAGCCATCAGGCCTGGGGATGGACCTCCCTGGAGCTGGGCATTCAAGAGGATGAGTGCAGCAGAACGCGTATCCAGGGAGGTCTGGTCTGCTGATTTTTCTAGGCATAAAGCAAATTTATTTTTTAAATTAAAATAAAAATGCATGCACTTTCAAATGTA',
 'metadata': {'biotype': 'lincRNA',
  'geneID': 'ENSG00000233866',
  'order': 0,
  'ranges': GenomicRanges(chr='22', start=16062237, end=16062396, id='ENSG00000233866', strand='+'),
  'transcriptID': 'ENST00000424770'}}

In [34]:
data_generator = DataLoader(data, batch_size=5, collate_fn=numpy_collate, shuffle=False)
data_generator = iter(data_generator)

In [35]:
data = next(data_generator)

In [36]:
data

{'inputs': array([ 'TAGCCATCAGGCCTGGGGATGGACCTCCCTGGAGCTGGGCATTCAAGAGGATGAGTGCAGCAGAACGCGTATCCAGGGAGGTCTGGTCTGCTGATTTTTCTAGGCATAAAGCAAATTTATTTTTTAAATTAAAATAAAAATGCATGCACTTTCAAATGTA',
        'GAATGTACAGACTAGCACAGAAGTTGGGCTAATTAAACAAGTATTGCTGAAAATGAGTGCTGTAGATGACATGAGAGCAGGTATGGGGTTGATTGTTAGGGAAGTATAACTTAAAAGTTTATAAAGTTTCACATACTTCTCTTTATATTCTATAGGTAAT',
        'TGGCAACTGTTTTATAGTCACATCATCGAAGTCCAAAGGAAAATGTTTTCAGCATTGTGTGAAATATGATTTTCAACCACGTAAGGTAGGTAAAAGTAAATATTTTTATAACTCACTTGTTATGACAGAATTCTTAACTATCCTTTTATGACCTCTGTAG',
        'ATACTCAACTCTGCCTGGAATCATCACCAAAAGAGGATGCATCAATTTTTGTGCATTCCCAACATGCTCTAATGCTTCCGGTGGGTGAATCATGGCTTTGTTTTCATGTTCTTGTCAGAATTTAACAATATTTTTATTTTATATATCAAACATTGCTCAT',
        'GCAGCGGCGTGGTCAGAGCGAGCTTCGGAGAAGCAGTGGTGGGTTCCATGTGATGGTGGAGTAGGAGGCAGGTCTCCGCGGTAAGTGGCGGGGGCGTGGACCCCACCGGGAACCCTCCCGGCTCCTTCCCTGCCTCTCCCTGTTTTCGTGCTTTCACTTC'],
       dtype='<U160'),
 'metadata': {'biotype': array(['lincRNA', 'pseudogene', 'pseudogene', 'pseudogene',
         'processed_

In [37]:
data['inputs']

array([ 'TAGCCATCAGGCCTGGGGATGGACCTCCCTGGAGCTGGGCATTCAAGAGGATGAGTGCAGCAGAACGCGTATCCAGGGAGGTCTGGTCTGCTGATTTTTCTAGGCATAAAGCAAATTTATTTTTTAAATTAAAATAAAAATGCATGCACTTTCAAATGTA',
       'GAATGTACAGACTAGCACAGAAGTTGGGCTAATTAAACAAGTATTGCTGAAAATGAGTGCTGTAGATGACATGAGAGCAGGTATGGGGTTGATTGTTAGGGAAGTATAACTTAAAAGTTTATAAAGTTTCACATACTTCTCTTTATATTCTATAGGTAAT',
       'TGGCAACTGTTTTATAGTCACATCATCGAAGTCCAAAGGAAAATGTTTTCAGCATTGTGTGAAATATGATTTTCAACCACGTAAGGTAGGTAAAAGTAAATATTTTTATAACTCACTTGTTATGACAGAATTCTTAACTATCCTTTTATGACCTCTGTAG',
       'ATACTCAACTCTGCCTGGAATCATCACCAAAAGAGGATGCATCAATTTTTGTGCATTCCCAACATGCTCTAATGCTTCCGGTGGGTGAATCATGGCTTTGTTTTCATGTTCTTGTCAGAATTTAACAATATTTTTATTTTATATATCAAACATTGCTCAT',
       'GCAGCGGCGTGGTCAGAGCGAGCTTCGGAGAAGCAGTGGTGGGTTCCATGTGATGGTGGAGTAGGAGGCAGGTCTCCGCGGTAAGTGGCGGGGGCGTGGACCCCACCGGGAACCCTCCCGGCTCCTTCCCTGCCTCTCCCTGTTTTCGTGCTTTCACTTC'],
      dtype='<U160')

In [38]:
HAL = HALModel()

In [39]:
HAL.predict_on_batch(next(data_generator))

array([ 16.58693109,  11.91657679,  13.674915  ,  15.40467572,  12.35613842])